# Insurance Risk Analytics — Exploratory Data Analysis
Dataset: `MachineLearningRating_v3/MachineLearningRating_v3.txt` (pipe-delimited, Feb 2014 - Aug 2015).

## Notebook checklist
1. Load the raw TXT (via DVC) and confirm schema/shape.
2. Produce descriptive statistics, missing-value audit, and loss-ratio KPIs.
3. Generate three story-telling visuals (province risk, claim distribution, temporal trend).

In [1]:
# Core imports
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style='whitegrid')
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.eda import basic_summary, loss_ratio

In [2]:
# Load AlphaCare data (pipe-delimited)
DATA_PATH = project_root / 'MachineLearningRating_v3' / 'MachineLearningRating_v3.txt'
assert DATA_PATH.exists(), f'Data file not found: {DATA_PATH}'
raw_df = pd.read_csv(
    DATA_PATH,
    sep='|',
    parse_dates=['TransactionMonth'],
    low_memory=False
)
print(f'Shape: {raw_df.shape}')
raw_df.head()

Shape: (1000098, 52)


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


In [ ]:
# Basic structure and quality checks
numeric_desc = raw_df.describe().T
categorical_card = raw_df.select_dtypes(include='object').nunique().sort_values(ascending=False)
missing = raw_df.isnull().sum()

print('Top 15 numerical fields by variance:')
display(numeric_desc.sort_values('std', ascending=False).head(15))

print('\nTop 15 categorical fields by cardinality:')
display(categorical_card.head(15))

print('\nLargest missing-value counts:')
missing[missing > 0].sort_values(ascending=False).head(20)

In [ ]:
# Portfolio KPIs
portfolio_loss_ratio = loss_ratio(raw_df)
total_policies = raw_df['PolicyID'].nunique() if 'PolicyID' in raw_df else len(raw_df)
print(f'Total policies: {total_policies:,}')
print(f'Portfolio loss ratio: {portfolio_loss_ratio:.2%}')

In [ ]:
# Plot 1 — Loss ratio by province
plt.figure(figsize=(10, 5))
province_metrics = (raw_df.groupby('Province')[['TotalClaims', 'TotalPremium']].sum()
                    .assign(LossRatio=lambda d: d['TotalClaims'] / d['TotalPremium']))
province_metrics = province_metrics.sort_values('LossRatio', ascending=False)
sns.barplot(x=province_metrics.index, y=province_metrics['LossRatio'], palette='magma')
plt.xticks(rotation=45, ha='right')
plt.title('Loss Ratio by Province')
plt.ylabel('Loss Ratio (TotalClaims / TotalPremium)')
plt.tight_layout()

In [ ]:
# Plot 2 — TotalClaims distribution and outliers
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
sns.histplot(raw_df['TotalClaims'], bins=40, kde=True, ax=axes[0], color='steelblue')
axes[0].set_title('TotalClaims Distribution')
axes[0].set_xlabel('TotalClaims (ZAR)')
sns.boxplot(x=raw_df['TotalClaims'], ax=axes[1], color='tomato')
axes[1].set_title('TotalClaims Outliers')
plt.tight_layout()

In [ ]:
# Plot 3 — Monthly trend of claims, premium, and loss ratio
df_time = raw_df.copy()
monthly = (df_time.set_index('TransactionMonth')
           .resample('M')[['TotalClaims', 'TotalPremium']].sum())
monthly['LossRatio'] = monthly['TotalClaims'] / monthly['TotalPremium']
fig, ax1 = plt.subplots(figsize=(12, 5))
ax1.plot(monthly.index, monthly['TotalClaims'], label='TotalClaims', color='crimson')
ax1.plot(monthly.index, monthly['TotalPremium'], label='TotalPremium', color='navy')
ax1.set_ylabel('Amount (ZAR)')
ax2 = ax1.twinx()
ax2.plot(monthly.index, monthly['LossRatio'], label='Loss Ratio', color='darkgreen', linestyle='--')
ax2.set_ylabel('Loss Ratio')
fig.legend(loc='upper right')
plt.title('Monthly Claims, Premiums, and Loss Ratio')
plt.tight_layout()

## Next steps
- Add commentary around segments (VehicleType, Gender, PostalCode) with notable lift in loss ratio.
- Feed findings into Task-1 report and design hypotheses for Task-3 tests.
- Version this notebook output via DVC or store cleaned aggregates for downstream modeling.